In [17]:
import pymysql

In [ ]:
con = pymysql.connect(
    host= 'localhost',
    port =  3306,
    user = 'root',
    password = '1234',
    db = 'mldb3',
    charset = 'utf8'
)
con
print('1. 연결성공')

In [ ]:
cur = con.cursor()
cur
print('2. 커서 객체 획득')

In [ ]:
#sql = 'select * from member'
sql = """
    select o2.orderid, m.name , o2.productid , o2.totalprice
    from orderlist o2
    inner join `member` m 
    on o2.memberid = m.id and m.id = 'park'
    order by productid desc
    """

In [ ]:
result_count = cur.execute(sql)
print('3. sql전송 성공', result_count)

In [51]:
rows=cur.fetchall()
print('4. 검색결과 --------')
print(rows)

4. 검색결과 --------
(('apple', 'java', 'fun java', '555555555'), ('park', 'jsp', 'fun jsp', '888'), ('apple', 'spring', 'fun spring', '555555555'), ('apple', 'android', 'fun android', '555555555'), ('park', 'swift', 'fun swift', '888'), ('summer', 'python', 'fun hahaha', 'summer'), ('park', 'jsp', 'fun jsp', '888'), ('park', 'jsp', 'fun jsp2', '888'), ('apple', 'java', 'good java', '555555555'))


In [52]:
con.close() # 메모리가 잡혀있는 상태이기때문에 끊어줘야한다

In [27]:
for index, row, in enumerate(rows):
    #print(index,row)
    print(index,row)

0 alicia
1 apple
2 apple111
3 com
4 park
5 pen
6 spring
7 summer
8 sun
9 sunday2
10 water
11 win
12 win2
13 win3
14 winter


### 함수로 만들기

In [128]:
def connect():
    con = pymysql.connect(
    host='localhost',
    port=3306,
    user='root',
    password='1234',
    db='mldb',
    charset='utf8'
    )
    print('1. 연결성공')
    cur = con.cursor()
    print('2. 커서 객체 획득')
    return cur,con

return을 넣어야 메모리값에만 있지않고 변수가 넣어진다. 즉 겉값에들어가지않는다

In [129]:
#위에 함수에 리턴 넣은데로 순서대로 작성해야 값이 들어감
cur,con=connect()

1. 연결성공
2. 커서 객체 획득


In [80]:
def sql_execute(cur,data):
    sql = """
    select o2.orderid, m.name , o2.productid , o2.totalprice
    from orderlist o2
    inner join `member` m 
    on o2.memberid = m.id and m.id = %s
    order by productid desc
    """
    result= cur.execute(sql,data)
    return result

In [73]:
sql_result =sql_execute(cur,'apple')
sql_result

4

1. 누가 어떤 물건을 얼마나 샀는가? 

In [103]:
who = input('누가??:')

누가??:park


In [104]:
sql_result2 = sql_execute(cur, who)
sql_result2

4

In [83]:
def out_data(cur):
    rows =cur.fetchall()
    return rows

In [105]:
result_rows = out_data(cur)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000))

In [85]:
def db_close(con):
    con.close()

In [106]:
# con 연결 끊는거 위에서 했던거 처럼 함수화시컨것뿐
db_close(con)

2.제일 높은 금액은?

In [94]:
[x[3] for x in result_rows]

[2000, 4000]

In [95]:
max([x[3] for x in result_rows])

4000

3.제일 많이 산 물건의 id는?

In [96]:
import pandas as pd

In [107]:
result_df = pd.DataFrame(data = result_rows, 
                         columns = ['주문id', 'name', '제품id', '주문총가격'])

In [108]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000


In [109]:
[x for x in result_df['제품id']]

['101', '100', '100', '100']

In [100]:
from collections import Counter

In [110]:
counter_id = Counter(result_df['제품id'])
counter_id

Counter({'101': 1, '100': 3})

In [111]:
max_id = counter_id.most_common(n=1)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 3)]
100


'100'

In [112]:
def sql_execute2(cur, data):
    sql = """
    select * from product
    where id = %s
    """
    result2 = cur.execute(sql, data)
    return result2

In [113]:
cur, con = connect()
cur

1. 연결성공
2. 커서 객체 획득


In [114]:
sql_result2 = sql_execute2(cur, max_id_result)
sql_result2

1

In [115]:
result_rows = out_data(cur)
result_rows[0]

('100', 'java', 'fun java', '1000')

In [116]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('-----------------------------------')
print('물건의 이름은 ', result_rows[0][1])
print('물건의 내용은 ', result_rows[0][2])
print('물건의 가격은 ', result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
-----------------------------------
물건의 이름은  java
물건의 내용은  fun java
물건의 가격은  1000


In [117]:
def sql_execute3(con, cur, data):
    sql = """
    insert into orderlist values (null, %s, %s, %s)
    """
    result3 = cur.execute(sql, data)
    con.commit()
    return result3

In [118]:
order_question = input('구매하시겠습니까?(yes/no)')
if order_question == 'yes':
    print('-------구매를 진행합니다.---------')
    your_id = input('당신의 id는 ')
    data_list = [your_id, result_rows[0][0], int(result_rows[0][3])]
    result4 = sql_execute3(con, cur, data_list)
    if result4 == 1:
        print('----구매가 완료되었습니다.')
    else:
        print('----구매에 문제가 발생했습니다.')

구매하시겠습니까?(yes/no)yes
-------구매를 진행합니다.---------
당신의 id는 park
----구매가 완료되었습니다.


In [119]:
con.close()

In [120]:
### 추천을 하고자 하는 컨텐츠기반 룰을 정하고, 
### 물건을 3개 이상 추천하는 추천시스템을 완성하시오.
### db의 데이터를 임의로 추가할 수 있음.

In [160]:
cur,con=connect()

1. 연결성공
2. 커서 객체 획득


In [161]:
def sql_execute4(cur, data):
    sql = """
    select o.orderid ,o.productid
    from orderlist o 
    inner join product p 
    on o.productid = p.id 
    where o.memberid != %s
    """
    result4 = cur.execute(sql, data)
    return result4

In [162]:
who = input('누가?? :')

누가?? :park


In [163]:
sql_result4 = sql_execute4(cur, who)
sql_result4

21

In [164]:
def out_data(cur):
    rows = cur.fetchall()
    return rows

In [165]:
result_rows = out_data(cur)
result_rows

((5, '100'),
 (6, '101'),
 (7, '102'),
 (9, '101'),
 (10, '103'),
 (11, '104'),
 (12, '105'),
 (13, '101'),
 (14, '106'),
 (15, '104'),
 (16, '103'),
 (17, '105'),
 (18, '105'),
 (19, '105'),
 (20, '108'),
 (21, '109'),
 (22, '100'),
 (23, '101'),
 (24, '103'),
 (25, '108'),
 (26, '107'))

In [137]:
db_close(con)

In [166]:
result_df2 = pd.DataFrame(data = result_rows, 
                         columns = ['주문id',  '제품id'])

In [167]:
result_df2

,주문id,제품id
0,5,100
1,6,101
2,7,102
3,9,101
4,10,103
5,11,104
6,12,105
7,13,101
8,14,106
9,15,104


In [168]:
counter_id = Counter(result_df2['제품id'])

In [169]:
counter_id

Counter({'100': 2,
         '101': 4,
         '102': 1,
         '103': 3,
         '104': 2,
         '105': 4,
         '106': 1,
         '108': 2,
         '109': 1,
         '107': 1})

In [ ]:
from collections import Counter

In [200]:
max_id = counter_id.most_common(n=3)
top3=[]
print(max_id)
for i in range(3) :
    max_id_result = max_id[i][0]
    top3.append(max_id_result)    

[('101', 4), ('105', 4), ('103', 3)]


In [201]:
top3

['101', '105', '103']

In [187]:
def sql_execute5(cur, data):
    sql = """
    select * from product
    where id = %s
    """
    result5 = cur.execute(sql, data)
    return result5

In [223]:
result_rows2=[]
for i in range(3):
    sql_result6 = sql_execute5(cur, top3[i])
    result_rows2.append(out_data(cur)) 
result_rows2   

[(('101', 'jsp', 'fun jsp', '2000'),),
 (('105', 'ios', 'fun ios', '6000'),),
 (('103', 'android', 'fun android', '4000'),)]

In [247]:
l1=result_rows2[0][0]
l2=result_rows2[1][0]
l3=result_rows2[2][0]

In [251]:
print('당신에게 추천하고 싶은 3가지 물건은 다음과 같습니다.')
print('-----------------------------------')
print('물건의 이름 ', l1[1],'가격 ' ,l1[3] ,'입니다.')
print('물건의 이름 ', l2[1],'가격 ' ,l2[3] ,'입니다.')
print('물건의 이름 ', l3[1],'가격 ' ,l3[3] ,'입니다.')

당신에게 추천하고 싶은 3가지 물건은 다음과 같습니다.
-----------------------------------
물건의 이름  jsp 가격  2000 입니다.
물건의 이름  ios 가격  6000 입니다.
물건의 이름  android 가격  4000 입니다.


In [265]:
def sql_execute6(con, cur, data):
    sql = """
    insert into orderlist values (null, %s, %s, %s)
    """
    result7 = cur.execute(sql, data)
    con.commit()
    return result7

In [271]:
def buy():
    order_question = input('구매하시겠습니까?(yes/no) :')
    if order_question == 'yes':
        print('-------구매를 진행합니다.---------')
        your_id = input('당신의 id는 ')
        to_buy = input('사고싶은 물건은?')
        to_buy_count = input('필요한 개수는?')
        if to_buy == 'jsp':
            to_buy = l1[0]
            to_buy_count = int(l1[3]) * int(to_buy_count)
        elif to_buy =='ios':
            to_buy =l2[0]
            to_buy_count = int(l2[3]) * int(to_buy_count) 
        elif to_buy == 'android':
            to_buy =l3[0]
            to_buy_count = int(l3[3]) * int(to_buy_count) 
        data_list = [your_id, to_buy,to_buy_count]
        result8 = sql_execute6(con, cur, data_list)
        if result8 == 1:
            print('----구매가 완료되었습니다.')
            print('총 주문금액:',to_buy_count)
        else:
            print('----구매에 문제가 발생했습니다.')

In [272]:
buy()

구매하시겠습니까?(yes/no) :yes
-------구매를 진행합니다.---------
당신의 id는 win
사고싶은 물건은?ios
필요한 개수는?3
----구매가 완료되었습니다.
총 주문금액: 18000
